In [1]:
#분류 활용
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tensorflow.keras.models import load_model
import time
import threading
from joblib import load

# 바이낸스 API 설정
api_key ='input_your_api_key'
api_secret = 'input_your_api_secret_key'
binance = ccxt.binance({
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True,
    'options': {'defaultType': 'future'}
})

model_path = r"C:\Users\admin\Desktop\model_epoch_4k_150_24_82.h5" ## input your model path
model = load_model(model_path)
look_back = 24
scaler_path = r"C:\Users\admin\Desktop\coinScale\4k_scaler.save" ## input your scaler path
scaler = load(scaler_path)

input_queue = []
stop_program = False
open_position_price = None


# 거래량 설정 (USDT 기준)
amount_in_usdt = int(input("Enter your amount - USDT: "))
leverage = int(input("Enter your leverage: "))  # leverage set
stop_loss = float(input("Enter your stop_loss: "))  # stop loss set(%)

current_position = None  # 

def set_stop_loss(symbol, position_size, last_price, is_long, stop_loss):
    if is_long:
        stop_price = last_price * (1 - stop_loss)  # 롱 포지션의 경우 가격 하락 시 정지 손실
    else:
        stop_price = last_price * (1 + stop_loss)  # 숏 포지션의 경우 가격 상승 시 정지 손실

    
    params = {
        'stopPrice': stop_price,
        'closePosition': True 
    }

    if is_long:
        binance.create_order(symbol, 'STOP_MARKET', 'SELL', position_size, None, params)
    else:
        binance.create_order(symbol, 'STOP_MARKET', 'BUY', position_size, None, params)
    print(f"Set stop-loss at {stop_price}")

def get_position_size(symbol):
    symbol = symbol.replace("/", "")
    positions = binance.fapiPrivateV2GetPositionRisk({'symbol': symbol})
    position_size = 0
    for position in positions:
        if position['symbol'] == symbol:
            position_size = abs(float(position['positionAmt']))
            break
    return position_size

def cancel_stop_loss_orders(symbol):
    global binance
    try:
        open_orders = binance.fetch_open_orders(symbol=symbol)
        for order in open_orders:
            if order['type'] == 'stop_market':
                binance.cancel_order(order['id'], symbol=symbol)
        print("Stop loss orders cancelled successfully.")
    except Exception as e:
        print(f"Error cancelling stop loss orders: {e}")

def close_position(symbol, last_price):
    global current_position, open_position_price
    position_size = get_position_size(symbol)
    if current_position == 'long':
        profit_loss = (last_price - open_position_price) * position_size
        binance.create_market_sell_order(symbol, position_size)
    elif current_position == 'short':
        profit_loss = (open_position_price - last_price) * position_size
        binance.create_market_buy_order(symbol, position_size)
    
    print(f"Profit/Loss: {profit_loss} USDT")
    current_position = None
    open_position_price = None 
def get_input(input_queue):
    while True:
        inp = input()
        input_queue.append(inp)

def predict_and_trade():
    global current_position, open_position_price 
    symbol = 'BTC/USDT'
    timeframe = '1h'
    limit = 20 + look_back
    
    since_time = datetime.utcnow() - timedelta(hours=limit)
    since = binance.parse8601(since_time.isoformat() + 'Z')

    candles = binance.fetch_ohlcv(symbol, timeframe, since, limit=limit)
    df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['returns'] = df['close'].pct_change().fillna(0)
    features = df[['close', 'returns','high','low']].values[-look_back:] 

    # 스케일링 및 예측
    features_scaled = scaler.transform(features)
    current_batch = features_scaled.reshape((1, look_back, 4))
    predicted_return = model.predict(current_batch)[0][0]
    print(f"Predicted return: {predicted_return}")

    last_price = df['close'].iloc[-1]
    total_usdt_with_leverage = amount_in_usdt * leverage
    amount_in_coin = total_usdt_with_leverage / last_price
    
    market_info = binance.market(symbol)
    amount_precision = market_info['precision']['amount']
    amount_in_coin = round(amount_in_coin, amount_precision)
    
    if predicted_return > 0.5 and current_position != 'long':  # 상승 예측이고 현재 롱 포지션 아닌 경우
        now = datetime.now()
        print("time: ", now.strftime("%Y-%m-%d %H:%M:%S"))
        if current_position == 'short':
            close_position(symbol, last_price)  # 현재 숏 포지션인 경우, 포지션 청산
            cancel_stop_loss_orders(symbol)
            open_position_price = last_price 
        else:
            open_position_price = last_price 
        print(f"Going long with {amount_in_coin} {symbol}")
        order = binance.create_market_buy_order(symbol, amount_in_coin)  # 롱 포지션 개설
        current_position = 'long'
        set_stop_loss(symbol, amount_in_coin, last_price, True, stop_loss)  # 롱 포지션에 대한 정지 손실 설정

    elif predicted_return <= 0.5 and current_position != 'short':  # 하락 예측이고 현재 숏 포지션 아닌 경우  
        now = datetime.now()
        print("time: ", now.strftime("%Y-%m-%d %H:%M:%S"))
        if current_position == 'long':
            close_position(symbol, last_price)  # 현재 롱 포지션인 경우, 포지션 청산
            cancel_stop_loss_orders(symbol)
            open_position_price = last_price 
        else:
            open_position_price = last_price
        print(f"Going short with {amount_in_coin} {symbol}")
        order = binance.create_market_sell_order(symbol, amount_in_coin)  # 숏 포지션 개설
        current_position = 'short'
        set_stop_loss(symbol, amount_in_coin, last_price, False, stop_loss)  # 숏 포지션에 대한 정지 손실 설정

def main():
    global input_queue
    input_thread = threading.Thread(target=get_input, args=(input_queue,))
    input_thread.daemon = True
    input_thread.start()

    while True:
        if input_queue and input_queue[-1] == 'q':
            print("Program is exiting...")
            break
            
        
        current_time = datetime.utcnow()
        wait_time = 3600 - (current_time.minute * 60 + current_time.second)
        print("waiting...\n\n")
        time.sleep(wait_time + 60)
        predict_and_trade()

if __name__ == "__main__":
    main()

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Enter your amount - USDT: 15
Enter your leverage: 10
Enter your stop_loss: 0.02
waiting...


1/1 [==============================] - 1s 725ms/step
Predicted return: 0.0006734876660630107
time:  2024-04-21 16:01:03
Going short with 0.002 BTC/USDT
Set stop-loss at 66327.03
waiting...


1/1 [==============================] - 0s 24ms/step
Predicted return: 0.29422464966773987
waiting...


1/1 [==============================] - 0s 24ms/step
Predicted return: 0.910274863243103
time:  2024-04-21 18:01:01
Profit/Loss: 0.229 USDT
Stop loss orders cancelled successfully.
Going long with 0.002 BTC/USDT
Set stop-loss at 63613.76
waiting...


1/1 [==============================] - 0s 24ms/step
Predicted return: 0.0002331572031835094
time:  2024-04-21 19:01:03
Profit/Loss: -0.10619999999999709 USDT
Stop loss orders cancelled successfully.
Going short with 0.002 BTC/USDT
Set stop-loss at 66156.07800000001
waiting...


1/1 [==============================] - 0s 16ms/step
Predicted return: 0.267451941967

InvalidOrder: binance amount of BTC/USDT:USDT must be greater than minimum amount precision of 3